In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
MODULE_PATH = '/home/ursho/PhD/Projects/Pteropods/My_parcels/Parcels_master_copy/parcels/parcels/__init__.py'
MODULE_NAME = "parcels"
import importlib
import sys
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

# Import all modules

In [3]:
from parcels import ParticleSet

In [10]:
sys.path.insert(1,"/net/kryo/work/ursho/PhD/Projects/Pteropod_IBM/Shelled_Pteropod_IBM/")
import spIBM

In [13]:
import Project_files

In [14]:
import os
import datetime
import xarray as xr

# Read in yaml file and user inputs

In [19]:
year, version, control, config_file = 1984, 2, 0, "IBM_config_parameters.yaml"
My_config = spIBM.read_config_files(config_file)

# Define fieldset

In [16]:
fieldset = spIBM.read_environment(My_config, year, control)

Adding lower and upper bounds...


# Check if the initial population needs to be calculated and test it

In [20]:
flag_calculate_initial_population = My_config.flag_calculate_initial_population
print(flag_calculate_initial_population)
directory_mort = My_config.directory_mort
similarity_file = My_config.similarity_file
output_dir = My_config.output_dir_initialization
output_dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


False


In [21]:
np.random.seed(seed=My_config.seed)
my_pteropods = spIBM.define_initial_population(number_of_individuals=1500, start_generation=0,number_of_attributes=17)



In [ ]:
flag = False
if flag == True:
    runIBM_module.run_IBM_idealized(My_config,my_pteropods,start_gen=0,time=5000,L_t=None,save_population=True,save_abundance=True)

# Determine the starting day

In [ ]:
ref_data_file ="/home/ursho/PhD/Projects/Pteropod_IBM/Data/MarEDat20120203Pteropods.nc"
daily_abundance_maredat, std_abundance_maredat = Project_files.get_daily_maredat_obs(ref_data=ref_data_file)



In [ ]:
output_dir = My_config.output_dir_initialization
gen0_file = My_config.gen0_file
gen1_file = My_config.gen1_file
observations = daily_abundance_maredat
observations_std = std_abundance_maredat


start_day = initialization_module.determine_starting_day(output_dir,gen0_file,gen1_file,observations,observations_std,start=None)
start_day

In [ ]:
initial_population = np.genfromtxt(output_dir+'/Pteropods_Day_{}.csv'.format(int(start_day)),delimiter=',')
num_init = initial_population.shape[0]
num_init

# Get initial positions

In [ ]:

grid_file = My_config.mesh_file
outfile = My_config.output_dir_initialization+My_config.initial_positions_file

np.random.seed(seed=My_config.version*5)

flag = True
#On the first year, calculate the initial positions. Later read them from file
if flag == True:
    latlon_list = Project_files.get_initial_positions(num=num_init,grid_file=grid_file,outfile=outfile)

    
latlon_list = np.genfromtxt(outfile, delimiter=',')

In [ ]:
pclass = parcels_module.PteropodParticle
ptero_pset = initialization_module.initialize_particles(fieldset,pclass,initial_population,latlon_list)
ptero_pset

# Run physics only for X days

In [ ]:
kernel = ptero_pset.Kernel(parcels_module.pteropod_kernel)

In [ ]:
flag = False
if flag == True:
    ptero_pset = runIBM_module.run_physics_only(My_config, ptero_pset, fieldset, kernel, year, total_runtime=3, dt=1.0, outputdt=1.0)
    
#always read from file. On the first year 1984 calculate the value and then read from file
my_file = My_config.output_dir_physics+My_config.physics_only_file.format(My_config.version)
ptero_pset = ParticleSet.from_particlefile(fieldset=fieldset, pclass=pclass, filename=my_file,lonlatdepth_dtype=np.float32)


In [ ]:
ptero_pset = initialization_module.reset_particle_attributes(ptero_pset,initial_population)
ptero_pset

# Start real run

In [ ]:
print('Starting simulation...')

next_ID = max(initial_population[:,0])+1
print('Shape initial:', initial_population.shape)
print('Next ID is {}'.format(next_ID))
current_gen = np.nanmax(initial_population[np.squeeze(np.argwhere((initial_population[:,2]==3) | (initial_population[:,3] == max(np.unique(initial_population[:,3]))))).astype(int),1])


In [ ]:
d0 = datetime.date(year,1,1)
d1 = datetime.date(year,1,10)
time_mat = np.empty((3,(d1-d0).days))
for i in range(time_mat.shape[1]):
    time_mat[0,i] = (d0+datetime.timedelta(days=i)).year
    time_mat[1,i] = (d0+datetime.timedelta(days=i)).day
    time_mat[2,i] = i

In [ ]:

runIBM_module.run_IBM_coupled(My_config, ptero_pset, fieldset, pclass, kernel, time_mat, next_ID, current_gen, L_t=None)

# Compare to original to see if the number of individuals and position is more or less the same

In [ ]:
file_mort_new = "/home/ursho/PhD/Projects/Pteropod_IBM/Analysis/Pteropod_Areareseed/output/mortalities/year_{}_V_2_control_0/Mortality_Day_9.csv".format(year)
file_mort_orig = "/home/ursho/PhD/Projects/Pteropod_IBM/Analysis/output_simulation_independent_V4/mortalities/year_{}_V_2_control_0/Mortality_Day_9.csv".format(year)

mort_orig = np.genfromtxt(file_mort_orig,delimiter=',')
mort_new = np.genfromtxt(file_mort_new,delimiter=',')


In [ ]:
mort_orig[1,:]

In [ ]:
mort_new[1,:]

In [ ]:
file_orig = "/home/ursho/PhD/Projects/Pteropod_IBM/Analysis/output_simulation_independent_V4/netcdf/year_2000_V_2/JitPtero_Day_1.nc"
file_new = "/home/ursho/PhD/Projects/Pteropod_IBM/Analysis/Pteropod_Areareseed/output/netcdf/year_2000_V_2/JitPtero_Day_1.nc"


ds_orig = xr.open_dataset(file_orig)
ds_new = xr.open_dataset(file_new)

In [ ]:
ds_orig.z.values[:,-1]-ds_new.z.values[:,-1]